In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit

In [2]:
spark = SparkSession.builder.master('local').appName('Campaign').getOrCreate()

In [3]:
df_input = spark.read.options(header='True', InferSchema='True',  quote = "\"", escape = "\"").csv('events.csv')
df_input.count()

387567

In [47]:
##filtered records where customer having all 4 events
df_input.createOrReplaceTempView("event")
df_event_customer_count = spark.sql("select city ,campaign,customer_user_id,count(distinct event_name) as count from  event where event_name ='app_launch' OR event_name ='add_to_cart' OR event_name ='create_order' OR event_name ='payment_success' group by city,campaign,customer_user_id  having count = 4 order by city,campaign  ")
df_event_customer_count.show(10)

+----------------+--------------------+--------------------+-----+
|            city|            campaign|    customer_user_id|count|
+----------------+--------------------+--------------------+-----+
|           Abbur|App Installs_iOS_BLR|d44a70bc-967f-4d3...|    4|
|        Adilabad|App Installs_Andr...|659a7a18-14a6-46f...|    4|
|        Adilabad|ET-004-TOF-LAL-Pu...|d1483dbf-efcb-4e9...|    4|
|        Adilabad|TOI_BLRTIMES_DR25...|1de1a83e-a305-471...|    4|
|      Adraspalli|ET-005-TOF-Broad-...|37029dee-b8e1-495...|    4|
|            Agra|                  NA|8462e155-04e7-457...|    4|
|            Agra|                None|583ce738-356e-461...|    4|
|Airforce Academy|ET-001-App-Purcha...|168ce030-497a-44c...|    4|
|Airforce Academy|                  NA|ae54a5d1-a95b-4f8...|    4|
|Airforce Academy|                  NA|02138e31-30fa-49a...|    4|
+----------------+--------------------+--------------------+-----+
only showing top 10 rows



In [44]:
##total customer per campaign
df_event_customer_count.createOrReplaceTempView("event2")
df = spark.sql("select city,campaign ,count(distinct customer_user_id) as count from event2 where campaign !='NA' and campaign!='None' group by city,campaign order by count desc")
df.show(10)

+---------+--------------------+-----+
|     city|            campaign|count|
+---------+--------------------+-----+
|Bangalore|App Purchase_Andr...|  387|
|Hyderabad|ET-001-App-Purcha...|  184|
|Bangalore|App Installs_Andr...|  113|
|Bangalore|Website Download ...|  100|
|Bangalore|App Installs_iOS_BLR|   89|
|Hyderabad|ET-004-TOF-LAL-Pu...|   64|
|Hyderabad|App Installs_Andr...|   60|
|Bangalore|TOI_BangaloreTime...|   45|
|Bangalore|ET-005-ATC-Androi...|   42|
|Hyderabad|App Installs_iOS_HYD|   37|
+---------+--------------------+-----+
only showing top 10 rows

